In [3]:
# install COMET
!pip install git+https://github.com/Unbabel/COMET@main

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/Unbabel/COMET (to revision main) to /tmp/pip-req-build-r2qnxndb
  Running command git clone --filter=blob:none --quiet https://github.com/Unbabel/COMET /tmp/pip-req-build-r2qnxndb
  Running command git checkout -q main
  error: pathspec 'main' did not match any file(s) known to git
  error: subprocess-exited-with-error
  
  × git checkout -q main did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git checkout -q main did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [20]:
# using jupyter notebook is recommended
import os
print(os.listdir())
# if you use Google Colab add the files in the following way:
#  /content/Original_story.txt, /content/chatGPT_translation.txt, /content/GoogleTranslate_translation.txt
# /content/HumanTranslations/*.txt
# make sure the files are then referenced in the code as "/content/Original_story.txt" instead of "content/Original_story.txt"

['calculate_MATTR.py', 'verbeterdeversies.dart', 'verbeterdeversie2.txt', 'ST_MATTR.tsv', 'MT.tsv', 'HT.tsv', 'Untitled.ipynb', 'HumanTranslations', '__pycache__', 'LICENSE', 'ST.tsv', 'README.md', 'calculate_comet.ipynb:Zone.Identifier', 'to_txt.py', 'udstyle-master.zip', 'calculate_COMET.py', 'thesis_code.zip', 'Ten EN-NL human translations-20250512T083209Z-1-001.zip', 'content', 'udstyle.py', 'verbeterdeversie3', 'MT_MATTR.tsv', 'thesis_code', 'mattr_scores.tsv', '.ipynb_checkpoints', 'HT_MATTR.tsv', 'CITATION.cff', 'parse_with_spacy.py', 'SourceText', 'calculate_comet.ipynb', 'Original_story.conllu', 'Original_story.txt', 'MachineTranslations']


In [21]:
import regex as re
# opening all story files and splitting them after every sentence
# explanation of the regex expression: splits on every \n and on every .?!
# if there is a " or ” after the character, that is included in the sentence and it splits after that
with open("content/Original_story.txt", "r", encoding="utf-8") as f:
  ST = f.read()
  ST_sentences = re.split(r'(?<=[.!?]["”]?)\s+|\n+', ST)

with open("content/chatGPT_translation.txt", "r", encoding="utf-8") as f:
  GPT = f.read()
  GPT_sentences = re.split(r'(?<=[.!?]["”]?)\s+|\n+', GPT)

with open("content/GoogleTranslate_translation.txt", "r", encoding="utf-8") as f:
  GT = f.read()
  GT_sentences = re.split(r'(?<=[.!?]["”]?)\s+|\n+', GT)

# adding the source and translation texts into the right format
ST_GPT = []
for src, mt in zip(ST_sentences, GPT_sentences):
      ST_GPT.append({"src": src, "mt": mt})

ST_GT = []
for src, mt in zip(ST_sentences, GT_sentences):
  ST_GT.append({"src": src, "mt": mt})


In [6]:
from comet import download_model, load_from_checkpoint
model_qe_path = download_model("Unbabel/wmt20-comet-qe-da")
model = load_from_checkpoint(model_qe_path)
# calculating COMET for ChatGPT translation
ST_GPT_output = model.predict(ST_GPT, batch_size=8, gpus=1)
print (ST_GPT_output.system_score)

/home/nikkivgurp/.local/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/nikkivgurp/.local/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
2025-06-14 15:26:37.088096: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-14 15:26:37.417221: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-14 15:26:39.609461: W tensor

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Lightning automatically upgraded your loaded checkpoint from v1.3.5 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../.cache/huggingface/hub/models--Unbabel--wmt20-comet-qe-da/snapshots/2e7ffc84fb67d99cf92506611766463bb9230cfb/checkpoints/model.ckpt`
Encoder model frozen.
/home/nikkivgurp/.local/lib/python3.8/site-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100%|██████████████████████████████████████████████████████████| 10/10 [00:07<00:00,  1.29it/s]


0.3033279730497222


In [7]:
# calculating COMET for Google Translate translation
ST_GT_output = model.predict(ST_GT, batch_size=8, gpus=1)
print (ST_GT_output.system_score)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100%|██████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  2.53it/s]


0.30434252248194654


In [8]:
import os
HT_folder = 'content/HumanTranslations'
results = []
print(os.listdir(HT_folder))

# splitting the human translations based on their punctuation marks
for filename in sorted(os.listdir(HT_folder)):
  if filename.endswith('.txt'):
    filepath = os.path.join(HT_folder, filename)
    with open(filepath, "r", encoding="utf-8") as f:
        HT = f.read()
        HT_sentences = re.split(r'(?<=[.!?]["”’\']?)\s+|\n+', HT)
        
    ST_HT = []
    for src, mt in zip(ST_sentences, HT_sentences):
        ST_HT.append({"src": src, "mt": mt})
    print(filename, ST_HT)

['P10_NL.txt:Zone.Identifier', 'P6_NL.txt:Zone.Identifier', 'P6_NL.docx:Zone.Identifier', 'P9_NL.docx:Zone.Identifier', 'P6_NL.docx', 'P9_NL.txt', 'P7_NL.docx:Zone.Identifier', 'P3_NL.txt', 'P2_NL.docx:Zone.Identifier', 'P5_NL.conllu:Zone.Identifier', 'P10_NL.txt', 'P7_NL.conllu', 'P4_NL.conllu', 'P4_NL.docx', 'P1_NL.docx:Zone.Identifier', 'P3_NL.txt:Zone.Identifier', 'P2_NL.txt:Zone.Identifier', 'P2_NL.conllu', 'P4_NL.txt:Zone.Identifier', 'P7_NL.docx', 'P5_NL.conllu', 'P3_NL.conllu:Zone.Identifier', 'P10_NL.docx:Zone.Identifier', 'P8_NL.conllu', 'P1_NL.conllu', 'P7_NL.txt', 'P10_NL.conllu:Zone.Identifier', 'P5_NL.docx', 'P4_NL.txt', 'P7_NL.txt:Zone.Identifier', 'P5_NL.txt', 'P3_NL.docx', 'P9_NL.conllu:Zone.Identifier', 'P7_NL.conllu:Zone.Identifier', 'P8_NL.txt:Zone.Identifier', 'P10_NL.conllu', 'P9_NL.conllu', 'P8_NL.conllu:Zone.Identifier', 'P6_NL.txt', 'P2_NL.conllu:Zone.Identifier', 'P1_NL.conllu:Zone.Identifier', 'P4_NL.conllu:Zone.Identifier', 'P10_NL.docx', 'P8_NL.docx', 'P2_N

In [9]:
import os
HT_folder = 'content/HumanTranslations'
results = []
filename_result = []

for filename in sorted(os.listdir(HT_folder)):
  if filename.endswith('.txt'):
    filepath = os.path.join(HT_folder, filename)
    with open(filepath, "r", encoding="utf-8") as f:
        HT = f.read()
        HT_sentences = re.split(r'(?<=[.!?]["”’\']?)\s+|\n+', HT)

    ST_HT = []
    for src, mt in zip(ST_sentences, HT_sentences):
        ST_HT.append({"src": src, "mt": mt})
    # calculate COMET scores for human translations
    result = model.predict(ST_HT, batch_size=8, gpus=1)
    print(filename, result.system_score)
    filename_result.append([filename, result.system_score])
    results.append(result.system_score)
average = sum(results)/len(results)
print('the average: ', average)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100%|██████████████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.53it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


P10_NL.txt -0.09151611785943571


Predicting DataLoader 0: 100%|██████████████████████████████████████████████████████████| 10/10 [00:08<00:00,  1.20it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


P1_NL.txt 0.2717752243931356


Predicting DataLoader 0: 100%|██████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.84it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

P7_NL.txt -0.2956989386462067


Predicting DataLoader 0: 100%|██████████████████████████████████████████████████████████| 10/10 [00:08<00:00,  1.11it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


P8_NL.txt 0.1036485661507437


Predicting DataLoader 0: 100%|██████████████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.54it/s]


P9_NL.txt 0.21687073618369668
the average:  0.011821334338501884


In [10]:
# files that have to be modified, because they are below 0: P10, P3, P4, P7
# p4 modified by my so that the sentences align better, to test if the scores are higher
p4_modified = [
  {"src": "\ufeffPeter.", "mt": "Peter"},
  {"src": "“No, Antone, I have told thee many times, no, thou shalt not sell it until I am gone.”", "mt": "‘Nee, Antone, ik heb al meerdere malen gezegd, zolang ik leef, mag je hem niet verkopen.’"},
  {"src": "“But I need money; what good is that old fiddle to thee?", "mt": "‘Maar ik kan het geld goed gebruiken. Bovendien, wat heeft u nog aan die oude viool?"},
  {"src": "The very crows laugh at thee when thou art trying to play.", "mt": "Zelfs de kraaien krassen het uit als u probeert te spelen."},
  {"src": "Thy hand trembles so thou canst scarce hold the bow.", "mt": "Uw hand trilt zo erg dat u de strijkstok nauwelijks kunt vasthouden."},
  {"src": "Thou shalt go with me to the Blue to cut wood to-morrow.", "mt": "Morgen gaan we naar het bos om hout te kappen."},
  {"src": "See to it thou art up early.”", "mt": "Zorg dat u vroeg opstaat."},
  {"src": "“What, on the Sabbath, Antone, when it is so cold?", "mt": "Toch niet op de Sabbat, Antone, en dat met deze kou?’"},
  {"src": "I get so very cold, my son, let us not go to-morrow.”", "mt": "‘Dat is me te koud, jongen, laten we morgen thuisblijven."},
  {"src": "“Yes, to-morrow, thou lazy old man.", "mt": "We gaan morgen, oude luilak."},
  {"src": "Do not I cut wood upon the Sabbath?", "mt": "Ziet u mij uitrusten op de Sabbat?"},
  {"src": "Care I how cold it is?", "mt": "Maak ik mij soms zorgen om de kou?"},
  {"src": "Wood thou shalt cut, and haul it too, and as for the fiddle, I tell thee I will sell it yet.”", "mt": "Hout zult u hakken, en sjouwen, en wat betreft die viool, die ga ik verkopen, dat zeg ik u maar alvast.’"},
  {"src": "Antone pulled his ragged cap down over his low heavy brow, and went out.", "mt": "Antone trok zijn oude hoed wat verder over zijn borstelige wenkbrauwen en liep naar buiten."},
  {"src": "The old man drew his stool up nearer the fire, and sat stroking his violin with trembling fingers and muttering, “Not while I live, not while I live.”", "mt": "De oude man trok zijn stoel wat dichter bij het vuur, streelde met bevende vingers zijn viool en mompelde, ‘Over mijn lijk, over mijn lijk.’"},
  {"src": "Five years ago they had come here, Peter Sadelack, and his wife, and oldest son Antone, and countless smaller Sadelacks, here to the dreariest part of south-western Nebraska, and had taken up a homestead.", "mt": "Vijf jaar geleden waren ze hier gekomen, Peter Sadelack, zijn vrouw, zijn oudste zoon Antone en een heel stel kleine Sadelackjes, naar het aller druilerigste gedeelte van zuidwest Nebraska, waar ze waren neergestreken."},
  {"src": "Antone was the acknowledged master of the premises, and people said he was a likely youth, and would do well.", "mt": "VAntone werd aangewezen als de man des huizes, en men vond hem een veelbelovende vent."},
  {"src": "That he was mean and untrustworthy every one knew, but that made little difference.", "mt": "Hij was gemeen en onbetrouwbaar, dat wist iedereen, maar dat leek niet uit te maken."},
  {"src": "His corn was better tended than any in the county, and his wheat always yielded more than other men’s.", "mt": "Niemand zorgde ten slotte zo goed voor zijn maïs en hij wist altijd meer tarwe te verbouwen dan de andere mannen."},
  {"src": "Of Peter no one knew much, nor had any one a good word to say for him.", "mt": "Men wist niet veel over Peter, noch was men over hem te spreken."},
  {"src": "He drank whenever he could get out of Antone’s sight long enough to pawn his hat or coat for whiskey.", "mt": "Hij dronk zodra hij lang genoeg uit het zicht van Antone kon ontsnappen om zijn hoed of jas voor whisky te verpachten."},
  {"src": "Indeed there were but two things he would not pawn, his pipe and his violin.", "mt": "Er waren maar twee zaken die hij weigerde weg te doen: zijn pijp en zijn viool."},
  {"src": "He was a lazy, absent minded old fellow, who liked to fiddle better than to plow, though Antone surely got work enough out of them all, for that matter.", "mt": "Hij was een luie, dromerige oude man, die liever op zijn viool speelde dan meehielp op het land, hoewel Anton iedereen aan het werk zette."},
  {"src": "In the house of which Antone was master there was no one, from the little boy three years old, to the old man of sixty, who did not earn his bread.", "mt": "In Antone’s huishouden was er niemand die niet zwoegde voor de kost, van de kleine knaap van drie tot de oude man van zestig."},
  {"src": "Still people said that Peter was worthless, and was a great drag on Antone, his son, who never drank,  and was a much better man than his father had ever been.", "mt": "Toch vond men Peter maar waardeloos, en een blok aan het been van Antone, zijn zoon, die nooit dronk, en een veel betere man was dan zijn vader ooit was geweest."},
  {"src": "Peter did not care what people said.", "mt": "Het liet Peter koud wat de mensen van hem dachten."},
  {"src": "He did not like the country, nor the people, least of all he liked the plowing.", "mt": "Hij hield niet van het land, en de mensen, en al helemaal niet van werken op het land."},
  {"src": "He was very homesick for Bohemia.", "mt": "Hij verlangde terug naar de Bohemen."},
  {"src": "Long ago, only eight years ago by the calendar, but it seemed eight centuries to Peter, he had been a second violinist in the great theatre at Prague.", "mt": "Lang geleden, volgens de kalender acht jaar maar voor Peter leek het wel acht eeuwen, was hij de tweede violist in het Stadstheater van Praag."},
  {'src': 'He had gone into the theatre very young, and had been there all his life, until he had a stroke of paralysis, which made his arm so weak that his bowing was uncertain.', 'mt': 'Hij had zich al jong bij het gezelschap aangesloten, en had daar zijn hele leven doorgebracht, tot hij door een beroerte verlamd raakte, waardoor zijn arm zo zwak was geworden dat hij de viool niet meer kon bespelen.'},
  {'src': 'Then they told him he could go.', 'mt': 'Toen stuurden ze hem weg.'},
  {'src': 'Those were great days at the theatre.', 'mt': 'Hij had er een prachtige tijd gehad.'},
  {'src': 'He had plenty to drink then, and wore a dress coat every evening, and there were always parties after the play.', 'mt': 'Er was altijd genoeg te drinken, hij droeg elke avond een nette jas, en er werd na elk optreden gefeest.'},
  {'src': 'He could play in those days, ay, that he could!', 'mt': 'Hij speelde in die tijd de sterren van de hemel, och, wat kon hij spelen!'},
  {'src': 'He could never read the notes well, so he did not play first; but his touch, he had a touch indeed, so Herr Mikilsdoff, who led the orchestra, had said.', 'mt': 'Hij had nooit goed noten leren lezen, daarom was hij de tweede violist; maar hij speelde met gevoel, met enorm veel gevoel, althans dat zei Herr Mikilsdoff, de dirigent van het orkest.'},
  {'src': 'Sometimes now Peter thought he could plow better if he could only bow as he used to.', 'mt': 'Soms dacht Peter dat het werken op het land hem minder zwaar zou vallen als hij nog steeds zo kon spelen als vroeger.'},
  {'src': 'He had seen all the lovely women in the world there, all the great singers and the great players.', 'mt': 'Hij had daar alle prachtige vrouwen in de wereld gezien, alle beroemde zangers en acteurs.'},
  {'src': 'He was in the orchestra when Rachel played, and he heard Liszt play when the Countess d’Agoult sat in the stage box and threw the master white lilies.', 'mt': 'Hij zat in het orkest toen Rachel speelde, en hij hoorde Liszt spelen toen de Comtesse d’Agoult in de zaal zat en de maestro witte lelies toewierp.'},
  {'src': 'Once, a French woman came and played for weeks, he did not remember her name now.', 'mt': 'Dan was er nog die Franse vrouw die er enkele weken speelde. Hij was haar naam vergeten.'},
  {'src': 'He did not remember her face very well either, for it changed so, it was never twice the same.', 'mt': 'Hij herinnerde zich haar gezicht niet goed meer, aangezien die elke avond veranderde en geen twee keer hetzelfde was.'},
  {'src': 'But the beauty of it, and the great hunger men felt at the sight of it, that he remembered.', 'mt': 'Maar de schoonheid ervan, en het brandende verlangen dat de mannen voelden zodra ze dat gezicht zagen, dat herinnerde hij zich nog goed.'},
  {'src': 'Most of all he remembered her voice.', 'mt': 'Haar stem herinnerde hij zich nog het best.'},
  {'src': 'He did not know French, and could not understand a word she said, but it seemed to him that she must be talking the music of Chopin.', 'mt': 'Hij verstond geen Frans, en begreep geen woord van wat ze zei, maar het leek alsof ze sprak met de muziek van Chopin.'},
  {'src': 'And her voice, he thought he should know that in the other world.', 'mt': 'En haar stem, die zou hij in het hiernamaals nog herkennen.'},
  {'src': 'The last night she played a play in which a man touched her arm, and she stabbed him.', 'mt': 'De laatste avond speelde ze in een stuk waarin een man haar arm aanraakte, en zij hem vervolgens neerstak.'},
  {'src': 'As Peter sat among the smoking gas jets down below the footlights with his fiddle on his knee, and looked up at her, he thought he would like to die too, if he could touch her arm once, and have her stab him so.', 'mt': 'Peter zat tussen de rij gaslampjes die samen het voetlicht vormden, met zijn viool op zijn knie, en keek op naar haar, en op dat moment had hij ook wel willen sterven, als hij haar arm maar mocht aanraken, en zij hem daarna zou neersteken.'},
  {'src': 'Peter went home to his wife very drunk that night.', 'mt': 'Peter ging die avond erg dronken weer terug naar zijn vrouw.'},
  {'src': 'Even in those days he was a foolish fellow, who cared for nothing but music and pretty faces.', 'mt': 'Hij was zelfs in die dagen al een domme dwaas, die alleen maar bezig was met muziek en mooie vrouwen.'},
  {'src': 'It was all different now.', 'mt': 'Nu was alles anders.'},
  {'src': 'He had nothing to drink and little to eat, and here, there was nothing but sun, and grass, and sky.', 'mt': 'Er was niets te drinken en weinig te eten en dit land bestond alleen uit zon, gras en lucht.'},
  {'src': 'He had forgotten almost everything, but some things he  remembered well enough.', 'mt': 'Hij was bijna alles vergeten, maar sommige dingen herinnerde hij zich nog goed.'},
  {'src': 'He loved his violin and the holy Mary, and above all else he feared the Evil One, and his son Antone.', 'mt': 'Hij hield van zijn viool en de heilige Maria, en boven alles was hij als de dood voor de Duivel en zijn zoon Antone.'},
  {'src': 'The fire was low, and it grew cold.', 'mt': 'Het vuur begon te smeulen en het werd steeds kouder.'},
  {'src': 'Still Peter sat by the fire remembering.', 'mt': 'Toch bleef Peter bij het vuur zitten mijmeren over vroeger.'},
  {'src': 'He dared not throw more cobs on the fire; Antone would be angry.', 'mt': 'Hij durfde geen kooltjes meer op het vuur te gooien, dan werd Antone kwaad.'},
  {'src': 'He did not want to cut wood tomorrow, it would be Sunday, and he wanted to go to mass.', 'mt': 'Hij wilde morgen niet mee om hout te hakken, het was zondag en hij wilde naar de mis.'},
  {'src': 'Antone might let him do that.', 'mt': 'Misschien mocht dat wel van Antone.'},
  {'src': 'He held his violin under his wrinkled chin, his white hair fell over it, and he began to play “Ave Maria.”', 'mt': 'Hij legde zijn viool onder zijn gerimpelde kin en zijn witte haar viel over het instrument. Hij begon ‘Ave Maria’ te spelen.'},
  {'src': 'His hand shook more than ever before, and at last refused to work the bow at all.', 'mt': 'Zijn hand trilde meer dan ooit tevoren, tot hij het instrument helemaal niet meer kon bespelen.'},
  {'src': 'He sat stupefied for a while, then arose, and taking his violin with him, stole out into the old sod stable.', 'mt': 'Hij bleef een tijdje zo zitten, stond toen op en slofte met zijn viool onder de arm naar de oude turfschuur.'},
  {'src': 'He took Antone’s shot-gun down from its peg, and loaded it by the moonlight which streamed in through the door.', 'mt': 'Hij haalde Antone’s jachtgeweer van de haak en laadde het met kogels in het straaltje maanlicht dat door de deur naar binnen viel.'},
  {'src': 'He sat down on the dirt floor, and leaned back against the dirt wall.', 'mt': 'Hij ging zitten op de vieze vloer en leunde tegen de vieze muur.'},
  {'src': 'He heard the wolves howling in the distance, and the night wind screaming as it swept over the snow.', 'mt': 'Hij hoorde de wolven huilen in de verte en de wind schreeuwde in de nacht en raasde over de sneeuwvlakte.'},
  {'src': 'Near him he heard the regular breathing of the horses in the dark.', 'mt': 'Vlakbij hoorde hij de regelmatige ademhaling van de paarden in het donker.'},
  {'src': 'He put his crucifix above his heart, and folding his hands said brokenly all the Latin he had ever known, “Pater noster, qui in cælum est.”', 'mt': 'Hij hield zijn crucifix voor zijn hart, vouwde zijn handen en sprak houterig het enige Latijn dat hij ooit had gekend, ‘Pater noster, qui in cælum est.’.'},
  {'src': 'Then he raised his head and sighed, “Not one kreutzer will Antone pay them to pray for my soul, not one kreutzer, he is so careful of his money, is Antone, he does not waste it in drink, he is a better man than I, but hard sometimes.', 'mt': 'Toen hief hij zijn hoofd en verzuchtte, ‘Geen kreuzer zal Antone ze geven om te bidden voor mijn ziel, geen enkele kreuzer, hij is zo spaarzaam met zijn geld, die Antone, hij verkwist het niet aan drank, hij is een betere man dan ik ooit ben geweest, maar soms te hardvochtig.'},
  {'src': 'He works the girls too hard, women were not made to work so.', 'mt': 'Hij laat de meiden te hard werken, daar zijn vrouwen niet voor gemaakt.'},
  {'src': 'But he shall not sell thee, my fiddle, I can play thee no more, but they shall not part us.', 'mt': 'Maar hij zal u niet verkopen, mijn lieve viool, ik kan u niet meer bespelen, maar ze zullen ons niet van elkaar scheiden.'},
  {'src': 'We have seen it all together, and we will forget it together, the French woman and all.”', 'mt': 'We hebben samen alles gezien, en samen zullen we alles vergeten, de Franse vrouw en al het andere.’'},
  {'src': 'He held his fiddle under his chin a moment, where it had lain so often, then put it across his knee and broke it through the middle.', 'mt': 'Hij hield zijn viool even onder zijn kin, waar die al zo vaak had gelegen, en legde het instrument toen over zijn knie en brak hem doormidden.'},
  {'src': 'He pulled off his old boot, held the gun between his knees with the muzzle against his forehead, and pressed the trigger with his toe.', 'mt': 'Hij trok zijn oude laars uit, zette het geweer tussen zijn knieën met de loop tegen zijn voorhoofd, en haalde met zijn grote teen de trekker over.'},
  {'src': 'In the morning Antone found him stiff, frozen fast in a pool of blood.', 'mt': 'In de ochtend vond Antone hem, verstijfd en bevroren in een plas bloed.'},
  {'src': 'They could not straighten him out enough to fit a coffin, so they buried him in a pine box.', 'mt': 'Ze kregen hem niet meer rechtop genoeg om in een kist te passen, dus werd hij begraven in een vierkante kist van dennenhout.'},
  {'src': 'Before the funeral Antone carried to town the fiddle-bow which Peter had forgotten to break.', 'mt': 'Voor de begrafenis ging Antone naar de stad met de strijkstok die Peter vergeten was te breken.'},
  {'src': 'Antone was very thrifty, and a better man than his father had been.', 'mt': 'Antone wist overal een slaatje uit te slaan, en was een betere man dan zijn vader ooit was geweest.'},
  {'src': 'Willa Cather (1892)', 'mt': 'Willa Cather (1892)'}]

In [11]:
p4_modified_output = model.predict(p4_modified, batch_size=8, gpus=1)
print (p4_modified_output.system_score)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100%|██████████████████████████████████████████████████████████| 10/10 [00:07<00:00,  1.37it/s]


0.20940523467173702


In [12]:
p10_modified =  [{'src': '\ufeffPeter.', 'mt': 'Peter.'},
  {'src': '“No, Antone, I have told thee many times, no, thou shalt not sell it until I am gone.”', 'mt': '‘Neen, Antone, neen, ik heb het u menigmaal gezegd, ge zult hem niet verkopen, althans niet zolang ik nog in leven ben.’'},
  {'src': '“But I need money; what good is that old fiddle to thee?', 'mt': '‘Maar ik heb geld nodig; wat hebt gij aan die oude viool?'},
  {'src': 'The very crows laugh at thee when thou art trying to play.', 'mt': 'Zelfs de kraaien lachen u uit wanneer ge erop probeert te spelen.'},
  {'src': 'Thy hand trembles so thou canst scarce hold the bow.', 'mt': 'Uw hand trilt zo hevig dat ge de strijkstok nauwelijks kunt vasthouden.'},
  {'src': 'Thou shalt go with me to the Blue to cut wood to-morrow.', 'mt': 'Ge gaat morgen met mij mee naar The Blue om hout te hakken.'},
  {'src': 'See to it thou art up early.”', 'mt': 'Zorg dat ge vroeg uit de veren bent.’'},
  {'src': '“What, on the Sabbath, Antone, when it is so cold?', 'mt': '‘Meent ge dat nou, Antone, op zondag, met die kou?'},
  {'src': 'I get so very cold, my son, let us not go to-morrow.”', 'mt': 'Ik zou bijkans bevriezen, m’n jongen, laat ons morgen niet gaan.’'},
  {'src': '“Yes, to-morrow, thou lazy old man.', 'mt': '‘Jawel, morgen, gij oude luiaard.'},
  {'src': 'Do not I cut wood upon the Sabbath?', 'mt': 'Hak ik niet zelf ook hout op zondag?'},
  {'src': 'Care I how cold it is?', 'mt': 'Maal ik erom hoe koud het is?'},
  {'src': 'Wood thou shalt cut, and haul it too, and as for the fiddle, I tell thee I will sell it yet.”', 'mt': 'Ge zult hout hakken, en het sjouwen, en wat die viool betreft, ik zeg u dat ik hem ga verkopen.’'},
  {'src': 'Antone pulled his ragged cap down over his low heavy brow, and went out.', 'mt': 'Antone trok zijn sjofele pet dieper over zijn ogen en ging naar buiten.'},
  {'src': 'The old man drew his stool up nearer the fire, and sat stroking his violin with trembling fingers and muttering, “Not while I live, not while I live.”', 'mt': 'De oude man schoof zijn kruk dichter naar het vuur, ging zitten, liet zijn trillende vingers over zijn viool glijden en mompelde: ‘Niet zo lang ik leef, niet zo lang ik leef.’'},
  {'src': 'Five years ago they had come here, Peter Sadelack, and his wife, and oldest son Antone, and countless smaller Sadelacks, here to the dreariest part of south-western Nebraska, and had taken up a homestead.', 'mt': 'Vijf jaar eerder waren ze hier naartoe gekomen, Peter Sadelack en zijn vrouw, en zijn oudste zoon Antone, en talloze kleinere Sadelacks, naar dit somberste deel van South-West Nebraska, en ze hadden er een bestaan opgebouwd.'},
  {'src': 'Antone was the acknowledged master of the premises, and people said he was a likely youth, and would do well.', 'mt': 'Antone werd door iedereen als het hoofd van de familie beschouwd en men zei dat hij een veelbelovende jongeman was, die het ver zou schoppen.'},
  {'src': 'That he was mean and untrustworthy every one knew, but that made little difference.', 'mt': 'Dat hij gemeen en onbetrouwbaar was wist iedereen, maar dat maakte weinig uit.'},
  {'src': 'His corn was better tended than any in the county, and his wheat always yielded more than other men’s.', 'mt': 'Zijn graan kreeg meer zorg dan welk graan ook, en zijn tarwe bracht meer op dan dat van wie ook.'},
  {'src': 'Of Peter no one knew much, nor had any one a good word to say for him.', 'mt': 'Men wist maar weinig over Peter en niemand had een goed woord voor hem over.'},
  {'src': 'He drank whenever he could get out of Antone’s sight long enough to pawn his hat or coat for whiskey.', 'mt': 'Hij dronk zodra hij ook maar even kans zag zich lang genoeg aan Antones blik te onttrekken om zijn hoed of jas te verpanden voor whisky.'},
  {'src': 'Indeed there were but two things he would not pawn, his pipe and his violin.', 'mt': 'In feite waren er maar twee dingen die hij nooit zou verpanden: zijn pijp en zijn viool.'},
  {'src': 'He was a lazy, absent minded old fellow, who liked to fiddle better than to plow, though Antone surely got work enough out of them all, for that matter.', 'mt': 'Hij was een luie, verstrooide oude man, die liever een viool vasthield dan een ploeg, al liet Antone ze echt allemaal de handen uit de mouwen steken.'},
  {'src': 'In the house of which Antone was master there was no one, from the little boy three years old, to the old man of sixty, who did not earn his bread.', 'mt': 'In het huis waar Antone de scepter zwaaide was er niemand, van de kleine jongen van drie tot de oude man van zestig, die niet werkte voor de kost.'},
  {'src': 'Still people said that Peter was worthless, and was a great drag on Antone, his son, who never drank,  and was a much better man than his father had ever been.', 'mt': 'Evengoed werd er gezegd dat Peter nergens voor deugde en dat hij alleen maar een last was voor Antone, zijn zoon, die zelf geen druppel dronk en die een betere man was dan zijn vader ooit was geweest.'},
  {'src': 'Peter did not care what people said.', 'mt': 'Het liet Peter koud wat de mensen zeiden.'},
  {'src': 'He did not like the country, nor the people, least of all he liked the plowing.', 'mt': 'Hij vond het allemaal maar niks, het landschap niet, de mensen niet en het ploegen al helemaal niet.'},
  {'src': 'He was very homesick for Bohemia.', 'mt': 'Hij had heel erge heimwee naar Bohemen.'},
  {'src': 'Long ago, only eight years ago by the calendar, but it seemed eight centuries to Peter, he had been a second violinist in the great theatre at Prague.', 'mt': 'Lang geleden, volgens de kalender niet meer dan acht jaar, maar voor Peters gevoel eerder acht eeuwen, was hij tweede violist geweest in het schitterende Theater van Praag.'},
  {'src': 'He had gone into the theatre very young, and had been there all his life, until he had a stroke of paralysis, which made his arm so weak that his bowing was uncertain.', 'mt': 'Hij was al op heel jonge leeftijd bij het theater terecht gekomen en was er zijn hele leven gebleven, totdat hij een beroerte had gekregen en nog maar zo weinig kracht in zijn arm had dat hij de strijkstok niet meer trefzeker kon hanteren.'},
  {'src': 'Then they told him he could go.', 'mt': 'Toen had hij te verstaan gekregen dat hij kon vertrekken.'},
  {'src': 'Those were great days at the theatre.', 'mt': 'Het waren de hoogtijdagen van het Theater geweest.'},
  {'src': 'He had plenty to drink then, and wore a dress coat every evening, and there were always parties after the play.', 'mt': 'Er was altijd drank in overvloed, hij droeg elke avond een rokkostuum en na afloop van de voorstelling was er altijd wel een feest.'},
  {'src': 'He could play in those days, ay, that he could!', 'mt': 'Wat had hij mooi gespeeld in die tijd, echt de sterren van de hemel!'},
  {'src': 'He could never read the notes well, so he did not play first; but his touch, he had a touch indeed, so Herr Mikilsdoff, who led the orchestra, had said.', 'mt': 'Hij kon niet echt goed noten lezen, dus was hij nooit eerste violist geworden; maar hij had er gevoel voor, hij had er echt gevoel voor, dat had Herr Mikilsdoff, die het orkest leidde, zelf gezegd.'},
  {'src': 'Sometimes now Peter thought he could plow better if he could only bow as he used to.', 'mt': 'Peter dacht nu wel eens dat hij beter zou kunnen ploegen als hij de strijkstok nog zo zou weten te hanteren als vroeger.'},
  {'src': 'He had seen all the lovely women in the world there, all the great singers and the great players.', 'mt': 'Hij had daar de mooiste vrouwen ter wereld gezien, de beste zangers en de beste toneelspelers.'},
  {'src': 'He was in the orchestra when Rachel played, and he heard Liszt play when the Countess d’Agoult sat in the stage box and threw the master white lilies.', 'mt': 'Hij had in het orkest gezeten toen Rachel speelde, en hij had Liszt horen spelen terwijl gravin d’Agoult in de loge zat en de dirigent witte lelies toewierp.'},
  {'src': 'Once, a French woman came and played for weeks, he did not remember her name now.', 'mt': 'Er was een keer een Franse vrouw geweest die weken achtereen had gespeeld, hij kon zich haar naam niet meer herinneren.'},
  {'src': 'He did not remember her face very well either, for it changed so, it was never twice the same.', 'mt': 'Haar gezicht herinnerde hij zich ook niet goed omdat het zo veranderlijk was geweest, nooit twee keer hetzelfde.'},
  {'src': 'But the beauty of it, and the great hunger men felt at the sight of it, that he remembered.', 'mt': 'Maar de schoonheid van haar gezicht, en het hevige verlangen dat het opriep bij mannen, dat herinnerde hij zich nog goed.'},
  {'src': 'Most of all he remembered her voice.', 'mt': 'Vooral herinnerde hij zich haar stem.'},
  {'src': 'He did not know French, and could not understand a word she said, but it seemed to him that she must be talking the music of Chopin.', 'mt': 'Hij was het Frans niet machtig en hij verstond geen woord van wat ze zei, maar hij had het idee dat ze de muziek van Chopin sprak.'},
  {'src': 'And her voice, he thought he should know that in the other world.', 'mt': 'En dan haar stem. Hij vond dat hij die zou moeten horen in de andere wereld.'},
  {'src': 'The last night she played a play in which a man touched her arm, and she stabbed him.', 'mt': 'Ze had een keer gespeeld in een stuk waarin een man haar arm aanraakte, waarop zij hem had neergestoken met een mes.'},
 {'src': 'As Peter sat among the smoking gas jets down below the footlights with his fiddle on his knee, and looked up at her, he thought he would like to die too, if he could touch her arm once, and have her stab him so.', 'mt': 'Peter, die met zijn viool op zijn knie tussen de rookblazers onder de voetlichten zat, had naar haar opgekeken en bedacht dat hij ook zou willen sterven als hij maar een keertje haar arm mocht aanraken, waarna zij ook hem zou neersteken.'},
 {'src': 'Peter went home to his wife very drunk that night.', 'mt': 'Die avond kwam Peter stomdronken thuis bij zijn vrouw.'},
 {'src': 'Even in those days he was a foolish fellow, who cared for nothing but music and pretty faces.', 'mt': 'Zelfs toen al was hij een losbol, die enkel leefde voor muziek en mooie vrouwen.'},
 {'src': 'It was all different now.', 'mt': 'Dat was nu allemaal wel anders.'},
 {'src': 'He had nothing to drink and little to eat, and here, there was nothing but sun, and grass, and sky.', 'mt': 'Hij had geen drank en maar weinig eten en er was hier niets dan zon, gras en lucht.'},
 {'src': 'He had forgotten almost everything, but some things he  remembered well enough.', 'mt': 'Hij was vrijwel alles vergeten, maar sommige dingen herinnerde hij zich maar al te goed.'},
 {'src': 'He loved his violin and the holy Mary, and above all else he feared the Evil One, and his son Antone.', 'mt': 'Hij hield van zijn viool en van de maagd Maria, maar bovenal vreesde hij de Boze, en zijn zoon Antone.'},
 {'src': 'The fire was low, and it grew cold.', 'mt': 'Het vlammen doofden uit en het begon koud te worden.'},
 {'src': 'Still Peter sat by the fire remembering.', 'mt': 'Maar Peter zat nog altijd bij het vuur met zijn herinneringen.'},
 {'src': 'He dared not throw more cobs on the fire; Antone would be angry.', 'mt': 'Hij durfde niet meer kolen op het vuur te gooien; Antone zou boos worden.'},
 {'src': 'He did not want to cut wood tomorrow, it would be Sunday, and he wanted to go to mass.', 'mt': 'Hij wilde morgen geen hout hakken, dan was het zondag en hij wilde naar de mis.'},
 {'src': 'Antone might let him do that.', 'mt': 'Misschien dat Antone hem dat zou toestaan.'},
 {'src': 'He held his violin under his wrinkled chin, his white hair fell over it, and he began to play “Ave Maria.”', 'mt': 'Hij legde zijn viool tegen zijn rimpelige kin, zijn witgrijze haar viel over zijn voorhoofd, en hij begon het ‘Ave Maria’ te spelen.'},
 {'src': 'His hand shook more than ever before, and at last refused to work the bow at all.', 'mt': 'Zijn hand trilde heviger dan ooit en was uiteindelijk niet langer in staat de strijkstok vast te houden.'},
 {'src': 'He sat stupefied for a while, then arose, and taking his violin with him, stole out into the old sod stable.', 'mt': 'Peter staarde een tijdje onthutst voor zich uit, stond toen op en sloop, met zijn viool, naar de oude plaggenstal.'},
 {'src': 'He took Antone’s shot-gun down from its peg, and loaded it by the moonlight which streamed in through the door.', 'mt': 'Hij haalde Antones oude geweer van de haak en laadde het bij het maanlicht dat door de deur naar binnen viel.'},
 {'src': 'He sat down on the dirt floor, and leaned back against the dirt wall.', 'mt': 'Hij ging op de lemen vloer zitten en liet zijn rug tegen de lemen wand rusten.'},
 {'src': 'He heard the wolves howling in the distance, and the night wind screaming as it swept over the snow.', 'mt': 'In de verte hoorde hij de wolven huilen en de nachtelijke wind joeg met een ijselijk geluid over de sneeuw.'},
 {'src': 'Near him he heard the regular breathing of the horses in the dark.', 'mt': 'Vlak naast zich hoorde hij de regelmatige ademhaling van de paarden in het donker.'},
 {'src': 'He put his crucifix above his heart, and folding his hands said brokenly all the Latin he had ever known, “Pater noster, qui in cælum est.”', 'mt': 'Hij legde zijn kruisbeeldje op zijn hart, vouwde zijn handen en zei al het gebroken Latijn dat hij machtig was: ‘Pater noster, qui in cælum est.’'},
 {'src': 'Then he raised his head and sighed, “Not one kreutzer will Antone pay them to pray for my soul, not one kreutzer, he is so careful of his money, is Antone, he does not waste it in drink, he is a better man than I, but hard sometimes.', 'mt': 'Toen hief hij zijn hand en verzuchtte: ‘Antone zal hun niet één kreutzer betalen om te bidden voor mijn ziel, niet één kreutzer, hij zit zo boven op zijn geld, Antone, hij verkwist het niet aan drank, hij is een betere man dan ik, maar soms ook hard.'},
 {'src': 'He works the girls too hard, women were not made to work so.', 'mt': 'Hij is te hard voor de meisjes, vrouwen zijn niet gemaakt voor dat soort werk.'},
 {'src': 'But he shall not sell thee, my fiddle, I can play thee no more, but they shall not part us.', 'mt': 'Maar jou zal hij niet verkopen, mijn dierbare viool, ik kan je niet langer bespelen maar ze zullen ons niet van elkaar scheiden.'},
 {'src': 'We have seen it all together, and we will forget it together, the French woman and all.”', 'mt': 'We hebben het allemaal samen meegemaakt, en we zullen het ook samen vergeten, de Franse vrouw en al het overige.’'},
 {'src': 'He held his fiddle under his chin a moment, where it had lain so often, then put it across his knee and broke it through the middle.', 'mt': 'Hij hield zijn viool even onder zijn kin, waar hij zo vaak had gelegen, liet hem vervolgens op zijn knie rusten en brak hem doormidden.'},
 {'src': 'He pulled off his old boot, held the gun between his knees with the muzzle against his forehead, and pressed the trigger with his toe.', 'mt': 'Hij trok zijn oude laars uit, klemde het geweer tussen zijn knieën, met de loop tegen zijn voorhoofd, en haalde met zijn teen de trekker over.'},
 {'src': 'In the morning Antone found him stiff, frozen fast in a pool of blood.', 'mt': 'Zo trof Antone hem de volgende ochtend aan, stijf, vastgevroren in een plas bloed.'},
 {'src': 'They could not straighten him out enough to fit a coffin, so they buried him in a pine box.', 'mt': 'Ze konden hem niet meer recht genoeg krijgen om in een doodskist te passen, dus werd hij ter aarde besteld in een grenen opbergkist.'},
 {'src': 'Before the funeral Antone carried to town the fiddle-bow which Peter had forgotten to break.', 'mt': 'Nog voor de begrafenis ging Antone naar de stad met de strijkstok die Peter was vergeten doormidden te breken.'},
 {'src': 'Antone was very thrifty, and a better man than his father had been.', 'mt': 'Antone was zeer spaarzaam, en een betere man dan zijn vader was geweest.'},
 {'src': 'Willa Cather (1892)', 'mt': 'Willa Cather (1892)'}]

In [13]:
p10_modified_output = model.predict(p10_modified, batch_size=8, gpus=1)
print (p10_modified_output.system_score)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100%|██████████████████████████████████████████████████████████| 10/10 [00:07<00:00,  1.39it/s]


0.22234958699463228


In [14]:
p3_modified =  [{'src': '\ufeffPeter.', 'mt': 'Peter.'},
 {'src': '“No, Antone, I have told thee many times, no, thou shalt not sell it until I am gone.”', 'mt': '‘Nee, Antone, ik heb je al zo vaak gezegd: nee, zolang ik er nog ben, verkoop je hem niet.’'},
 {'src': '“But I need money; what good is that old fiddle to thee?', 'mt': '‘Maar ik heb geld nodig, en wat heb jij aan die oude viool?'},
 {'src': 'The very crows laugh at thee when thou art trying to play.', 'mt': 'Zelfs de kraaien lachen je uit wanneer je erop probeert te spelen.'},
 {'src': 'Thy hand trembles so thou canst scarce hold the bow.', 'mt': 'Je hand beeft zo erg dat je de strijkstok nauwelijks kunt vasthouden.'},
 {'src': 'Thou shalt go with me to the Blue to cut wood to-morrow.', 'mt': 'Je gaat morgen met me mee naar de Blue om hout te hakken.'},
 {'src': 'See to it thou art up early.”', 'mt': 'Zorg dat je op tijd op bent.’'},
 {'src': '“What, on the Sabbath, Antone, when it is so cold?', 'mt': '‘Op de rustdag, Antone, in deze kou?'},
 {'src': 'I get so very cold, my son, let us not go to-morrow.”', 'mt': 'Ik zal vernikkelen van de kou, mijn zoon. Laten we morgen niet gaan.’'},
 {'src': '“Yes, to-morrow, thou lazy old man.', 'mt': '‘Jawel, we gaan morgen, ouwe luilak die je bent.'},
 {'src': 'Do not I cut wood upon the Sabbath?', 'mt': 'Hak ik soms geen hout op de rustdag?'},
 {'src': 'Care I how cold it is?', 'mt': 'Kan het mij schelen hoe koud het is?'},
 {'src': 'Wood thou shalt cut, and haul it too, and as for the fiddle, I tell thee I will sell it yet.”', 'mt': 'Je gaat gewoon houthakken, en je gaat het versjouwen ook. En wat die viool betreft, ik zeg je, op een dag verkoop ik hem.'},
 {'src': 'Antone pulled his ragged cap down over his low heavy brow, and went out.', 'mt': 'Antone trok zijn sjofele pet diep over zijn lage, zware voorhoofd en ging naar buiten.'},
 {'src': 'The old man drew his stool up nearer the fire, and sat stroking his violin with trembling fingers and muttering, “Not while I live, not while I live.”', 'mt': 'De oude man schoof zijn kruk dichter bij het vuur, streek met trillende vingers over zijn viool en mompelde: ‘Niet zolang ik leef, niet zolang ik leef.’'},
 {'src': 'Five years ago they had come here, Peter Sadelack, and his wife, and oldest son Antone, and countless smaller Sadelacks, here to the dreariest part of south-western Nebraska, and had taken up a homestead.', 'mt': 'Ze waren hier vijf jaar geleden aangekomen, Peter Sadelack, zijn vrouw, hun oudste zoon Antone en een hele stoet jonge Sadelackjes, hier in het meest desolate deel van zuidwest Nebraska, waar ze een stuk land in gebruik hadden genomen.'},
 {'src': 'Antone was the acknowledged master of the premises, and people said he was a likely youth, and would do well.', 'mt': 'Antone werd als hoofd van het gezin beschouwd, en mensen zeiden dat hij een veelbelovende jongeman was die het ver zou schoppen.'},
 {'src': 'That he was mean and untrustworthy every one knew, but that made little difference.', 'mt': 'Dat hij gemeen en onbetrouwbaar was, wist iedereen, maar dat maakte weinig verschil.'},
 {'src': 'His corn was better tended than any in the county, and his wheat always yielded more than other men’s.', 'mt': 'Zijn mais werd het beste verzorgd van de hele streek, en zijn tarwe leverde altijd meer op dan die van anderen.'},
 {'src': 'Of Peter no one knew much, nor had any one a good word to say for him.', 'mt': 'Niemand wist veel van Peter, noch had iemand een goed woord voor hem over.'},
 {'src': 'He drank whenever he could get out of Antone’s sight long enough to pawn his hat or coat for whiskey.', 'mt': 'Hij dronk wanneer hij lang genoeg uit Antones zicht kon blijven om zijn hoed of jas te verpanden voor whisky.'},
 {'src': 'Indeed there were but two things he would not pawn, his pipe and his violin.', 'mt': 'Er waren maar twee dingen die hij weigerde te verpanden: zijn pijp en zijn viool.'},
 {'src': 'He was a lazy, absent minded old fellow, who liked to fiddle better than to plow, though Antone surely got work enough out of them all, for that matter.', 'mt': 'Hij was een luie, afwezige oude kerel, die liever vioolspeelde dan ploegde, hoewel Antone hen allemaal hard genoeg liet werken.'},
 {'src': 'In the house of which Antone was master there was no one, from the little boy three years old, to the old man of sixty, who did not earn his bread.', 'mt': 'In de hoeve waar Antone de scepter zwaaide, was er niemand, van het driejarige jongetje tot de oude man van zestig, die niet zijn brood verdiende.'},
 {'src': 'Still people said that Peter was worthless, and was a great drag on Antone, his son, who never drank,  and was a much better man than his father had ever been.', 'mt': 'Toch zeiden de mensen dat Peter een nietsnut was, een blok aan het been van Antone, zijn zoon, die nooit dronk en een veel beter man was dan zijn vader ooit was geweest.'},
 {'src': 'Peter did not care what people said.', 'mt': 'Peter trok zich niets aan van wat mensen zeiden.'},
 {'src': 'He did not like the country, nor the people, least of all he liked the plowing.', 'mt': 'Hij hield niet van het land, niet van de mensen, en het minst van alles hield hij van ploegen.'},
 {'src': 'He was very homesick for Bohemia.', 'mt': 'Hij had vreselijke heimwee naar Bohemen.'},
 {'src': 'Long ago, only eight years ago by the calendar, but it seemed eight centuries to Peter, he had been a second violinist in the great theatre at Prague.', 'mt': 'Lang geleden, slechts acht jaar volgens de kalender maar het voelde voor Peter als acht eeuwen, was hij tweede violist geweest in het grote theater van Praag.'},
 {'src': 'He had gone into the theatre very young, and had been there all his life, until he had a stroke of paralysis, which made his arm so weak that his bowing was uncertain.', 'mt': 'Hij was heel jong in het theater begonnen en er zijn hele leven gebleven, totdat hij een beroerte kreeg waardoor zijn arm zo zwak werd dat zijn stokvoering eronder leed.'},
 {'src': 'Then they told him he could go.', 'mt': 'Toen hadden ze hem gezegd dat hij kon vertrekken.'},
 {'src': 'Those were great days at the theatre.', 'mt': 'Het was een mooie tijd in het theater geweest.'},
 {'src': 'He had plenty to drink then, and wore a dress coat every evening, and there were always parties after the play.', 'mt': 'Hij had altijd genoeg te drinken, hij droeg elke avond een rokkostuum en na de voorstelling waren er altijd feestjes.'},
 {'src': 'He could play in those days, ay, that he could!', 'mt': 'Hij kon geweldig spelen in die tijd, ja, dat kon hij!'},
 {'src': 'He could never read the notes well, so he did not play first; but his touch, he had a touch indeed, so Herr Mikilsdoff, who led the orchestra, had said.', 'mt': 'Hij had nooit goed noten kunnen lezen, daarom speelde hij geen eerste viool; maar zijn klank, hij had een prachtige klank, had Herr Mikilsdoff, de dirigent van het orkest, gezegd.'},
 {'src': 'Sometimes now Peter thought he could plow better if he could only bow as he used to.', 'mt': 'Soms dacht Peter weleens dat hij misschien beter zou kunnen ploegen als hij nog kon strijken zoals vroeger.'},
 {'src': 'He had seen all the lovely women in the world there, all the great singers and the great players.', 'mt': 'Hij had er alle mooie vrouwen van de wereld gezien, alle grote zangers en acteurs.'},
 {'src': 'He was in the orchestra when Rachel played, and he heard Liszt play when the Countess d’Agoult sat in the stage box and threw the master white lilies.', 'mt': 'Hij had in de orkestbak gezeten toen Rachel op het toneel stond en hij had Liszt horen spelen toen gravin d’Agoult in de loge zat en de meester witte lelies toewierp.'},
 {'src': 'Once, a French woman came and played for weeks, he did not remember her name now.', 'mt': 'Er was eens een Franse vrouw geweest die wekenlang had opgetreden, maar haar naam herinnerde hij zich niet meer.'},
 {'src': 'He did not remember her face very well either, for it changed so, it was never twice the same.', 'mt': 'Ook haar gezicht herinnerde hij zich niet goed, want dat veranderde voortdurend en was geen twee keer hetzelfde.'},
 {'src': 'But the beauty of it, and the great hunger men felt at the sight of it, that he remembered.', 'mt': 'Maar hoe mooi haar gezicht was, en de begeerte die de mannen beving als ze het zagen, dat herinnerde hij zich nog levendig.'},
 {'src': 'Most of all he remembered her voice.', 'mt': 'Het meeste van alles herinnerde hij zich haar stem.'},
 {'src': 'He did not know French, and could not understand a word she said, but it seemed to him that she must be talking the music of Chopin.', 'mt': 'Hij sprak geen Frans en begreep geen woord van wat ze zei, maar als ze sprak klonk het voor hem als de muziek van Chopin.'},
 {'src': 'And her voice, he thought he should know that in the other world.', 'mt': 'En haar stem, die zou hij in het hiernamaals nog herkennen.'},
 {'src': 'The last night she played a play in which a man touched her arm, and she stabbed him.', 'mt': 'Op de laatste avond had ze een stuk gespeeld waarin een man haar arm aanraakte en zij hem neerstak.'},
 {'src': 'As Peter sat among the smoking gas jets down below the footlights with his fiddle on his knee, and looked up at her, he thought he would like to die too, if he could touch her arm once, and have her stab him so.', 'mt': 'Toen Peter met zijn viool op zijn knie tussen de rokende gaslampen onder de voetlichten zat en naar haar opkeek, bedacht hij dat hij ook wel wilde sterven als hij maar één keer haar arm kon aanraken en dan door haar neergestoken werd.'},
 {'src': 'Peter went home to his wife very drunk that night.', 'mt': 'Peter ging die avond stomdronken terug naar zijn vrouw.'},
 {'src': 'Even in those days he was a foolish fellow, who cared for nothing but music and pretty faces.', 'mt': 'Zelfs in die tijd was hij al een dwaas die nergens om gaf dan om muziek en mooie vrouwen.'},
 {'src': 'It was all different now.', 'mt': 'Nu was alles anders.'},
 {'src': 'He had nothing to drink and little to eat, and here, there was nothing but sun, and grass, and sky.', 'mt': 'Hij had geen drank en weinig eten, en het enige wat je hier had, was zon, gras en lucht.'},
 {'src': 'He had forgotten almost everything, but some things he  remembered well enough.', 'mt': 'Hij was bijna alles vergeten, maar sommige dingen herinnerde hij zich nog goed.'},
 {'src': 'He loved his violin and the holy Mary, and above all else he feared the Evil One, and his son Antone.', 'mt': 'Hij hield van zijn viool en de heilige Maria, en boven alles vreesde hij de duivel, en zijn zoon Antone.'},
 {'src': 'The fire was low, and it grew cold.', 'mt': 'Het vuur was bijna gedoofd, en het werd koud.'},
 {'src': 'Still Peter sat by the fire remembering.', 'mt': 'Toch bleef Peter bij het vuur zitten, denkend aan vroeger.'},
 {'src': 'He dared not throw more cobs on the fire; Antone would be angry.', 'mt': 'Hij durfde geen nieuwe kolven op het vuur te gooien; Antone zou boos worden.'},
 {'src': 'He did not want to cut wood tomorrow, it would be Sunday, and he wanted to go to mass.', 'mt': 'Hij wilde morgen niet houthakken, want het was morgen zondag en hij wilde naar de mis.'},
 {'src': 'Antone might let him do that.', 'mt': 'Misschien zou Antone hem dat toestaan.'},
 {'src': 'He held his violin under his wrinkled chin, his white hair fell over it, and he began to play “Ave Maria.”', 'mt': 'Hij zette zijn viool onder zijn gerimpelde kin, zijn witte haren vielen eroverheen, en begon ‘Ave Maria’ te spelen.'},
 {'src': 'His hand shook more than ever before, and at last refused to work the bow at all.', 'mt': 'Zijn hand trilde heviger dan ooit en uiteindelijk lukte het strijken helemaal niet meer.'},
 {'src': 'He sat stupefied for a while, then arose, and taking his violin with him, stole out into the old sod stable.', 'mt': 'Verbijsterd bleef hij een tijdje zitten, stond toen op en sloop stilletjes met zijn viool naar de oude zodenschuur.'},
 {'src': 'He took Antone’s shot-gun down from its peg, and loaded it by the moonlight which streamed in through the door.', 'mt': 'Hij pakte Antones geweer van de haak en laadde het bij het licht van de maan dat door de deuropening naar binnen viel.'},
 {'src': 'He sat down on the dirt floor, and leaned back against the dirt wall.', 'mt': 'Hij ging op de aarden vloer zitten en leunde met zijn rug tegen de aarden muur.'},
 {'src': 'He heard the wolves howling in the distance, and the night wind screaming as it swept over the snow.', 'mt': 'In de verte hoorde hij de wolven huilen en de nachtwind gierend over de sneeuw jakkeren.'},
 {'src': 'Near him he heard the regular breathing of the horses in the dark.', 'mt': 'Naast zich hoorde hij het regelmatige ademen van de paarden in het donker.'},
 {'src': 'He put his crucifix above his heart, and folding his hands said brokenly all the Latin he had ever known, “Pater noster, qui in cælum est.”', 'mt': 'Hij legde zijn crucifix op zijn hart, vouwde zijn handen en sprak hortend al het Latijn dat hij kende: ‘Pater noster, qui in caelum est.’'},
 {'src': 'Then he raised his head and sighed, “Not one kreutzer will Antone pay them to pray for my soul, not one kreutzer, he is so careful of his money, is Antone, he does not waste it in drink, he is a better man than I, but hard sometimes.', 'mt': 'Toen hief hij zijn hoofd op en zuchtte: ‘Antone zal geen kreutzer betalen om voor mijn ziel te laten bidden, geen kreutzer. Hij is zo zuinig met zijn geld, Antone, hij verspilt het niet aan drank, hij is een beter man dan ik, maar soms is hij hardvochtig.'},
 {'src': 'He works the girls too hard, women were not made to work so.', 'mt': 'Hij laat de meisjes te hard werken, vrouwen zijn niet gemaakt om zo hard te werken.'},
 {'src': 'But he shall not sell thee, my fiddle, I can play thee no more, but they shall not part us.', 'mt': 'Maar jou, mijn viool, zal hij niet verkopen. Ik kan je niet meer bespelen, maar ze zullen ons niet scheiden.'},
 {'src': 'We have seen it all together, and we will forget it together, the French woman and all.”', 'mt': 'We hebben alles samen meegemaakt, en we zullen het samen vergeten, de Franse vrouw, alles.’'},
 {'src': 'He held his fiddle under his chin a moment, where it had lain so often, then put it across his knee and broke it through the middle.', 'mt': 'Hij legde zijn viool nog even onder zijn kin, waar hij zo vaak had gelegen, legde hem toen op zijn knie en brak hem doormidden.'},
 {'src': 'He pulled off his old boot, held the gun between his knees with the muzzle against his forehead, and pressed the trigger with his toe.', 'mt': 'Hij trok zijn oude laars uit, zette het geweer tussen zijn knieën met de loop tegen zijn voorhoofd, en haalde met zijn teen de trekker over.'},
 {'src': 'In the morning Antone found him stiff, frozen fast in a pool of blood.', 'mt': 'De volgende morgen vond Antone hem, verstijfd en vastgevroren in een plas bloed.'},
 {'src': 'They could not straighten him out enough to fit a coffin, so they buried him in a pine box.', 'mt': 'Ze kregen hem niet recht genoeg voor een doodskist, dus ze begroeven hem in een dennenhouten kist.'},
 {'src': 'Before the funeral Antone carried to town the fiddle-bow which Peter had forgotten to break.', 'mt': 'Nog voor de begrafenis ging Antone naar de stad met de strijkstok, die Peter was vergeten te breken.'},
 {'src': 'Antone was very thrifty, and a better man than his father had been.', 'mt': 'Antone was erg zuinig, en een beter man dan zijn vader was geweest.'},
 {'src': 'Willa Cather (1892)', 'mt': 'Willa Cather (1892)'}]

In [15]:
p3_modified_output = model.predict(p3_modified, batch_size=8, gpus=1)
print (p3_modified_output.system_score)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100%|██████████████████████████████████████████████████████████| 10/10 [00:07<00:00,  1.32it/s]


0.2788177699732937


In [16]:
p7_modified = [{'src': '\ufeffPeter.', 'mt': 'Peter.'},
 {'src': '“No, Antone, I have told thee many times, no, thou shalt not sell it until I am gone.”', 'mt': '‘Nee, Antone, ik heb het reeds dikwijls gezegd, nee, ge zult hem niet van de hand doen eer ik ben heengegaan.’'},
 {'src': '“But I need money; what good is that old fiddle to thee?', 'mt': '‘Maar ik heb het geld nodig; wat heeft ge nog aan die oude viool?'},
 {'src': 'The very crows laugh at thee when thou art trying to play.', 'mt': 'Wanneer ge erop probeert te spelen, lachen waarachtig de kraaien u uit.'},
 {'src': 'Thy hand trembles so thou canst scarce hold the bow.', 'mt': 'Het trillen van uw hand is zo hevig dat ge amper de strijkstok kunt vasthouden.'},
 {'src': 'Thou shalt go with me to the Blue to cut wood to-morrow.', 'mt': 'Morgen zult ge mij vergezellen naar de Blue, om hout te hakken.'},
 {'src': 'See to it thou art up early.”', 'mt': 'Zie erop toe dat ge vroeg ontwaakt.’'},
 {'src': '“What, on the Sabbath, Antone, when it is so cold?', 'mt': '‘Heden, op de sabbat, Antone, in deze kou?'},
 {'src': 'I get so very cold, my son, let us not go to-morrow.”', 'mt': 'Mijn zoon, ik raak zo vreselijk bevangen van de kou, laat ons een andere dag gaan.’'},
 {'src': '“Yes, to-morrow, thou lazy old man.', 'mt': '‘Nee, we gaan morgen, gij luie grijsaard.'},
 {'src': 'Do not I cut wood upon the Sabbath?', 'mt': 'Ik hak evenwel hout op de sabbat, nietwaar?'},
 {'src': 'Care I how cold it is?', 'mt': 'Hoort ge mij klagen over de kou?'},
 {'src': 'Wood thou shalt cut, and haul it too, and as for the fiddle, I tell thee I will sell it yet.”', 'mt': 'Hout zult ge hakken, en het eveneens verslepen. En wat betreft de viool, ik zeg het u, die ga ik verkopen.’'},
 {'src': 'Antone pulled his ragged cap down over his low heavy brow, and went out.', 'mt': 'Antone trok zijn versleten muts over zijn gefronste voorhoofd en liep de deur uit.'},
 {'src': 'The old man drew his stool up nearer the fire, and sat stroking his violin with trembling fingers and muttering, “Not while I live, not while I live.”', 'mt': 'De oude man zette zijn krukje dichter bij het vuur en zat daar vervolgens met trillende vingers over zijn viool te strelen, terwijl hij mompelde: ‘Niet zolang ik in leven ben, niet zolang ik in leven ben.’'},
 {'src': 'Five years ago they had come here, Peter Sadelack, and his wife, and oldest son Antone, and countless smaller Sadelacks, here to the dreariest part of south-western Nebraska, and had taken up a homestead.', 'mt': 'Vijf jaar geleden waren ze hier neergestreken, Peter Sadelack, zijn vrouw, zijn oudste zoon Antone en nog ontelbaar veel kleine Sadelacks, om hier, in het somberste stukje zuidwest-Nebraska, een boerenbestaan op te bouwen.'},
 {'src': 'Antone was the acknowledged master of the premises, and people said he was a likely youth, and would do well.', 'mt': 'Antone was de vermeende opziener van het landgoed; de mensen vonden hem een veelbelovende jongeman en geloofden dat hij het wel zou maken in het leven.'},
 {'src': 'That he was mean and untrustworthy every one knew, but that made little difference.', 'mt': 'Dat hij gemeen en onbetrouwbaar was, wist menigeen, maar dat haalde niet veel uit.'},
 {'src': 'His corn was better tended than any in the county, and his wheat always yielded more than other men’s.', 'mt': 'Zijn maïs werd van heel het gewest het best verzorgd en zijn tarweproductie lag hoger dan die van anderen.'},
 {'src': 'Of Peter no one knew much, nor had any one a good word to say for him.', 'mt': 'Van Peter wist men niet veel, noch hadden de mensen een goed woordje voor hem over.'},
 {'src': 'He drank whenever he could get out of Antone’s sight long enough to pawn his hat or coat for whiskey.', 'mt': 'Hij dronk zo gauw als hij lang genoeg uit Antones blikveld was om zijn hoed of zijn jas te verpanden voor whiskey.'},
 {'src': 'Indeed there were but two things he would not pawn, his pipe and his violin.', 'mt': 'Er waren echter twee zaken die hij nimmer zou verpanden: zijn pijp en zijn viool.'},
 {'src': 'He was a lazy, absent minded old fellow, who liked to fiddle better than to plow, though Antone surely got work enough out of them all, for that matter.', 'mt': 'Hij was een luie, afwezige oude kerel, die liever viool speelde dan ploegde, ofschoon Antone hen allemaal overigens voldoende aan het werk wist te zetten.'},
 {'src': 'In the house of which Antone was master there was no one, from the little boy three years old, to the old man of sixty, who did not earn his bread.', 'mt': 'In het huis dat Antone opzag was er niemand die níét zijn kost verdiende; niet het jongetje van drie jaar oud, noch de oude man van zestig.'},
 {'src': 'Still people said that Peter was worthless, and was a great drag on Antone, his son, who never drank,  and was a much better man than his father had ever been.', 'mt': 'Toch hielden de mensen vol dat Peter waardeloos was en dat hij een grote last was voor Antone, zijn zoon, die nooit een druppel dronk en die een veel beter man was dan zijn vader ooit was geweest.'},
 {'src': 'Peter did not care what people said.', 'mt': 'Peter gaf niets om wat de mensen over hem zeiden.'},
 {'src': 'He did not like the country, nor the people, least of all he liked the plowing.', 'mt': 'Hij was niet weg van het land, noch van de mensen, en nog het allerminst van het ploegen.'},
 {'src': 'He was very homesick for Bohemia.', 'mt': 'Hij verlangde hevig terug naar de Bohemen.'},
 {'src': 'Long ago, only eight years ago by the calendar, but it seemed eight centuries to Peter, he had been a second violinist in the great theatre at Prague.', 'mt': 'Lang geleden, slechts acht jaar volgens de kalender, al leken het voor Peter acht eeuwen, was hij de tweede violist in het Nationaal Theater van Praag.'},
 {'src': 'He had gone into the theatre very young, and had been there all his life, until he had a stroke of paralysis, which made his arm so weak that his bowing was uncertain.', 'mt': 'Hij had zich reeds op zeer jonge leeftijd bij het theater gevoegd, en was daar ook heel zijn leven gebleven, totdat hij na een beroerte gedeeltelijk verlamd raakte, waardoor zijn arm zo zwak werd dat zijn strijkspel twijfelachtig was geworden.'},
 {'src': 'Then they told him he could go.', 'mt': 'Toen wezen ze hem de deur.'},
 {'src': 'Those were great days at the theatre.', 'mt': 'Hij had een geweldige tijd gehad bij het theater.'},
 {'src': 'He had plenty to drink then, and wore a dress coat every evening, and there were always parties after the play.', 'mt': 'Destijds had hij nog zat genoeg te drinken, en droeg hij elke avond een rokkostuum, en na de voorstelling waren er altijd feesten.'},
 {'src': 'He could play in those days, ay, that he could!', 'mt': 'En wat kon hij spelen, zeg, in die dagen!'},
 {'src': 'He could never read the notes well, so he did not play first; but his touch, he had a touch indeed, so Herr Mikilsdoff, who led the orchestra, had said.', 'mt': 'Hij kon nooit goed noten lezen, daarom speelde hij niet als eerste; maar hij had zijn aanleg, jawel, zo zei Herr Mikilsdoff, die het orkest leidde.'},
 {'src': 'Sometimes now Peter thought he could plow better if he could only bow as he used to.', 'mt': 'Welnu dacht Peter soms dat hij beter zou kunnen ploegen als hij nog maar net zo kon buigen als vroeger.'},
 {'src': 'He had seen all the lovely women in the world there, all the great singers and the great players.', 'mt': 'In het theater had hij alle fantastische vrouwen van de wereld gezien, alle grote zangeressen en de grootste toneelspelers.'},
 {'src': 'He was in the orchestra when Rachel played, and he heard Liszt play when the Countess d’Agoult sat in the stage box and threw the master white lilies.', 'mt': 'Hij zat in het orkest toen Rachel speelde, en hij hoorde Liszt spelen toen de Gravin d’Agoult in de loge avant-scène zat en de concertmeester witte lelies toewierp.'},
 {'src': 'Once, a French woman came and played for weeks, he did not remember her name now.', 'mt': 'Er was eens een Franse dame geweest, die wekenlang optrad, van wie hij zich nu haar naam niet meer voor de geest kon halen.'},
 {'src': 'He did not remember her face very well either, for it changed so, it was never twice the same.', 'mt': 'Haar gezicht kon hij zich evenmin goed voor de geest halen, omdat het steeds zo veranderde, nooit twee keer hetzelfde.'},
 {'src': 'But the beauty of it, and the great hunger men felt at the sight of it, that he remembered.', 'mt': 'Maar de schoonheid ervan, en het grote verlangen dat mannen voelden wanneer ze het aanschouwden, dat herinnerde hij zich nog wel.'},
 {'src': 'Most of all he remembered her voice.', 'mt': 'Hij herinnerde zich vooral haar stem.'},
 {'src': 'He did not know French, and could not understand a word she said, but it seemed to him that she must be talking the music of Chopin.', 'mt': 'Hij was het Frans niet machtig en kon geen woord van wat ze zei verstaan, maar het leek hem dat ze de muziek van Chopin sprak.'},
 {'src': 'And her voice, he thought he should know that in the other world.', 'mt': 'En haar stem, die zou hij in het volgende leven nog herkennen.'},
 {'src': 'The last night she played a play in which a man touched her arm, and she stabbed him.', 'mt': 'Op de laatste avond speelde ze in een voorstelling waarin een man haar arm aanraakte en ze hem vervolgens neerstak.'},
 {'src': 'As Peter sat among the smoking gas jets down below the footlights with his fiddle on his knee, and looked up at her, he thought he would like to die too, if he could touch her arm once, and have her stab him so.', 'mt': 'Peter zat in de nevel van de gaslampen, onder het voetlicht, met zijn viool op zijn schoot naar haar op te kijken en bedacht zich dat hij ook wel wilde sterven, als dat betekende dat hij haar arm eens mocht aanraken, en dat hij zich dan door haar zou laten neersteken.'},
 {'src': 'Peter went home to his wife very drunk that night.', 'mt': 'Die nacht ging Peter erg dronken naar huis, terug naar zijn vrouw.'},
 {'src': 'Even in those days he was a foolish fellow, who cared for nothing but music and pretty faces.', 'mt': 'Zelfs in die dagen was hij een dwaze knaap, en gaf hij alleen maar om muziek en mooie gezichten.'},
 {'src': 'It was all different now.', 'mt': 'Welnu was alles anders.'},
 {'src': 'He had nothing to drink and little to eat, and here, there was nothing but sun, and grass, and sky.', 'mt': 'Hij had niets om te drinken en weinig om te eten, en hier was niets dan zon en gras en lucht.'},
 {'src': 'He had forgotten almost everything, but some things he  remembered well enough.', 'mt': 'Hij was bijna alles vergeten, maar sommige zaken herinnerde hij zich nog goed genoeg.'},
 {'src': 'He loved his violin and the holy Mary, and above all else he feared the Evil One, and his son Antone.', 'mt': 'Hij hield van zijn viool en van de Heilige Maria, en bovenal vreesde hij het Kwaad, en zijn zoon Antone.'},
 {'src': 'The fire was low, and it grew cold.', 'mt': 'Het vuur kwijnde en het werd koud.'},
 {'src': 'Still Peter sat by the fire remembering.', 'mt': 'Peter bleef niettemin bij het vuur aan vroeger denken.'},
 {'src': 'He dared not throw more cobs on the fire; Antone would be angry.', 'mt': 'Hij durfde niet meer maïskolven op het vuur te gooien, dan werd Antone boos.'},
 {'src': 'He did not want to cut wood tomorrow, it would be Sunday, and he wanted to go to mass.', 'mt': 'Hij wilde morgen geen hout hakken, dan was het zondag en wilde hij naar de mis.'},
 {'src': 'Antone might let him do that.', 'mt': 'Wellicht zou Antone het toestaan.'},
 {'src': 'He held his violin under his wrinkled chin, his white hair fell over it, and he began to play “Ave Maria.”', 'mt': 'Hij zette zijn viool tegen zijn gerimpelde kin, zijn witte haar viel eroverheen, en begon ‘Ave Maria’ te spelen.'},
 {'src': 'His hand shook more than ever before, and at last refused to work the bow at all.', 'mt': 'Zijn hand trilde meer dan ooit en kon op het eind de stok helemaal niet meer hanteren.'},
 {'src': 'He sat stupefied for a while, then arose, and taking his violin with him, stole out into the old sod stable.', 'mt': 'Stomverbaasd bleef hij even zitten, stond toen op en liep lieverlee de oude zodenstal in.'},
 {'src': 'He took Antone’s shot-gun down from its peg, and loaded it by the moonlight which streamed in through the door.', 'mt': 'Hij pakte Antones geweer van de haak en laadde het bij het maanlicht dat door de deur naar binnen scheen.'},
 {'src': 'He sat down on the dirt floor, and leaned back against the dirt wall.', 'mt': 'Hij ging op de grond zitten en leunde achterover tegen de aarden wal.'},
 {'src': 'He heard the wolves howling in the distance, and the night wind screaming as it swept over the snow.', 'mt': 'In de nacht hoorde hij in de verte wolven huilen en de gierende wind over de sneeuw razen.'},
 {'src': 'Near him he heard the regular breathing of the horses in the dark.', 'mt': 'In zijn buurt hoorde hij de regelmatige ademhaling van de paarden in het donker.'},
 {'src': 'He put his crucifix above his heart, and folding his hands said brokenly all the Latin he had ever known, “Pater noster, qui in cælum est.”', 'mt': 'Hij hield zijn kruisbeeldje vast voor zijn hart, vouwde zijn handen en sprak hakkelend al het Latijn dat hij ooit had gekend: ‘Pater noster, qui in cælum est.’'},
 {'src': 'Then he raised his head and sighed, “Not one kreutzer will Antone pay them to pray for my soul, not one kreutzer, he is so careful of his money, is Antone, he does not waste it in drink, he is a better man than I, but hard sometimes.', 'mt': 'Toen richtte hij zijn hoofd op en verzuchtte: ‘Antone heeft er geen enkele kreuzer voor over opdat de Heer mij zal behoeden, geen enkele kreuzer, hij is zo zuinig met zijn geld, die Antone, hij verspilt het niet aan drinken, hij is een beter man dan ik, maar streng is hij ook.'},
 {'src': 'He works the girls too hard, women were not made to work so.', 'mt': 'Hij zet de meiden te hard aan het werk, vrouwen zijn er niet voor gemaakt om zulk zwaar werk te verrichten.'},
 {'src': 'But he shall not sell thee, my fiddle, I can play thee no more, but they shall not part us.', 'mt': 'Maar ú zal hij niet verkopen, mijn viool, ik kan u niet langer bespelen, maar ze kunnen ons niet van elkaar scheiden.'},
 {'src': 'We have seen it all together, and we will forget it together, the French woman and all.”', 'mt': 'Samen hebben we alles gezien en samen zullen we alles vergeten, de Franse vrouw incluis.’'},
 {'src': 'He held his fiddle under his chin a moment, where it had lain so often, then put it across his knee and broke it through the middle.', 'mt': 'Hij legde zijn viool even tegen zijn kin, waar hij zo vaak had gelegen, zette hem toen op zijn knie en brak hem doormidden.'},
 {'src': 'He pulled off his old boot, held the gun between his knees with the muzzle against his forehead, and pressed the trigger with his toe.', 'mt': 'Hij trok zijn oude laars uit, hield het geweer klem tussen zijn knieën, zette de loop tegen zijn voorhoofd en drukte de trekker in met zijn teen.'},
 {'src': 'In the morning Antone found him stiff, frozen fast in a pool of blood.', 'mt': 'De volgende ochtend trof Antone hem stijf aan, vastgevroren in een plas bloed.'},
 {'src': 'They could not straighten him out enough to fit a coffin, so they buried him in a pine box.', 'mt': 'Ze konden hem niet voldoende strekken om hem in een doodskist te leggen, daarom begroeven ze hem in een grenen bak.'},
 {'src': 'Before the funeral Antone carried to town the fiddle-bow which Peter had forgotten to break.', 'mt': 'Voor de begrafenis plaatsvond nam Antone de strijkstok die Peter was vergeten te breken mee naar het dorp.'},
 {'src': 'Antone was very thrifty, and a better man than his father had been.', 'mt': 'Antone was uiterst zuinig, en een beter man dan zijn vader ooit was geweest.'},
 {'src': 'Willa Cather (1892)', 'mt': 'Willa Cather (1892)'}]

In [17]:
p7_modified_output = model.predict(p7_modified, batch_size=8, gpus=1)
print (p7_modified_output.system_score)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100%|██████████████████████████████████████████████████████████| 10/10 [00:07<00:00,  1.28it/s]


0.2590081767228089


In [18]:
print(filename_result)

[['P10_NL.txt', -0.09151611785943571], ['P1_NL.txt', 0.2717752243931356], ['P2_NL.txt', 0.24295566456490442], ['P3_NL.txt', -0.4051654791263373], ['P4_NL.txt', -0.44272585129855496], ['P5_NL.txt', 0.2338856229754655], ['P6_NL.txt', 0.2841839160476076], ['P7_NL.txt', -0.2956989386462067], ['P8_NL.txt', 0.1036485661507437], ['P9_NL.txt', 0.21687073618369668]]


In [19]:
# calculating mean with modified versions
sum_results = filename_result[1][1] + filename_result[2][1] + p3_modified_output.system_score + p4_modified_output.system_score + filename_result[5][1] + filename_result[6][1] + p7_modified_output.system_score + filename_result[7][1] + filename_result[8][1] + filename_result[9][1] + p10_modified_output.system_score

average = sum_results/10
print(average)

0.20272015600318188
